In [40]:
import shutil

source_file_path = '.\..\..\.env'

destination_file_path = '.'

shutil.copy(source_file_path, destination_file_path)

'.\\.env'

In [41]:
import django
import os
import sys
project_path = '../../'  # Adjust this to your actual project path
sys.path.append(project_path)
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'oogway.settings')
django.setup()

import re
from asgiref.sync import sync_to_async
from telethon.tl.types import  Message, PeerChannel
from telethon.tl.functions.messages import GetHistoryRequest
from telethon.sync import TelegramClient, events
from tqdm import tqdm
from datetime import datetime, timedelta, timezone
from PostAnalyzer.models import (
    Channel,
    EntryTarget,
    Market,
    Post,
    PostStatus,
    Predict,
    Symbol,
    TakeProfitTarget,
    SettingConfig,
    PositionSide,
    MarginMode,
)
import warnings
from django.forms.models import model_to_dict
from Shared.helpers import returnSearchValue,convertToJsonFile,subtractTime,load_json,print_colored
from Shared.findSameSignal import findSameSignal
from Shared.findShortOrderStat import findShortOrderStat
from Shared.findLongOrderStat import findLongOrderStat
from dotenv import dotenv_values
from Shared.findRiskToReward import findRiskToReward

config = dotenv_values(".env")

api_id = config["api_id"]
api_hash = config["api_hash"]

username = config["username"]

error_msg = []


allPredicts = []

historic_json_path = "./../historic-json"


# Disable all warnings
warnings.filterwarnings("ignore")

## is Predict Msg?

In [42]:
def isPredictMsg(msg):
    patterns = [
        r"📌 #(.+)",
        r"Entry:(.+)",
        r"Leverage:(.+)",
        r"Stop : (.+)",
        r"TP:(.+)",
    ]

    # Check if all patterns have a value
    return all(re.search(pattern, msg) for pattern in patterns)


## Symbol

In [43]:
async def findSymbol(msg):
   symbol = re.search(r"📌 #(.+)?USDT", msg, re.IGNORECASE)
   
   try:
      return await sync_to_async(Symbol.objects.get)(asset=returnSearchValue(symbol).upper())
   except:
      return None

## Market

In [44]:
async def findMarket(msg):

    market = None
    if "Futures Call".lower() in msg.lower():
        market = "FUTURES"
    elif "Spot".lower() in msg.lower():
        market = "SPOT"
    
    try:
        market_value = await sync_to_async(Market.objects.get)(name=market.upper())
        return market_value
    except:
        return None


## Position

In [45]:
async def findPosition(msg):
    if "short" in msg.lower():
        pos = "SHORT"
    elif "long" in msg.lower():
        pos = "LONG"

    try:
        market_value = await sync_to_async(PositionSide.objects.get)(name=pos)
        return market_value
    except:
        return None

## Leverage and Margin Mode

In [46]:
async def findLeverage(msg):
    leverage_match = re.search(r"Leverage:\s*(\d+x)", msg, re.IGNORECASE)
    if leverage_match:
        leverage_value = int(leverage_match.group(1).lower().replace("x","")) 
        # leverage_value = int(leverage_match.group(1)[2].lower().replace("x",""))   
    else:
        leverage_value = None

    leverage_type = await sync_to_async(MarginMode.objects.get)(name="ISOLATED")   
    
    return leverage_type, leverage_value

## StopLoss

In [47]:
def findStopLoss(msg):
   msg = msg.replace(",","")
   msg = msg.replace("‌","")
 
   return returnSearchValue(re.search(r"Stop\s*:\s*(?:کلوز)?\s*([\d.]+)", msg))

## Entry Targets

In [48]:
def findEntryTargets(msg):
    entry_targets_match = re.search(r"Entry:(.+?)\n\n", msg, re.DOTALL)
    entry_values = (
        re.findall(r"\d+(?:\.\d+)?", entry_targets_match.group(1))
        if entry_targets_match
        else None
    )
    return [float(x.strip()) for x in entry_values]

## Take Profits

In [49]:
def findTakeProfits(msg):
    take_profit_targets_match = re.search(r"TP:(.+?)\n\n", msg, re.DOTALL)
    profit_values = (
        re.findall(r"\d+(?:\.\d+)?", take_profit_targets_match.group(1))
        if take_profit_targets_match
        else None
    )
    return [float(x.strip()) for x in profit_values]
        

## Find important parts of a predict message such as symbol or entry point

In [50]:
async def predictParts(string, post):
    if string is None or post is None:
        return None
    
    try:
        should_check = True
        settings = await sync_to_async(SettingConfig.objects.get)(id=1)
        

        # symbol
        symbol_value:Symbol = await findSymbol(string)

        # market
        market_value= await findMarket(string)
        isSpot = market_value.name == "SPOT"

        # position, leverage, marginMode
        position_value = None
        leverage_value = None
        marginMode_value = None
        if not isSpot:
            position_value = await findPosition(string)
            marginMode_value, leverage_value = await findLeverage(string)
        else:
            position_value= await sync_to_async(PositionSide.objects.get)(name="BUY")
        
        # stopLoss
        stopLoss_value = findStopLoss(string)

        # entry targets
        entry_targets_value = findEntryTargets(string)

        # status    
        status_value = await sync_to_async(PostStatus.objects.get)(name="PENDING")

        # take_profit targets
        take_profit_targets_value = findTakeProfits(string)
        take_profit_targets_value = take_profit_targets_value[:8]

        # return Error RR < 0.2
        # TODO will add to setting, to get min RR
        RR = findRiskToReward(entry_targets_value, take_profit_targets_value, stopLoss_value)
        if RR < 0.2:
            status_value = await sync_to_async(PostStatus.objects.get)(name="ERROR")
            should_check = False

        # return false if signal already exists
        is_same_signal = await findSameSignal(post.date, symbol_value,market_value,position_value, leverage_value, marginMode_value,stopLoss_value,
                                    take_profit_targets_value,entry_targets_value,post.channel.id)
        if is_same_signal:
            return False
        
        print(symbol_value.name, position_value.name)
        isSHORT = position_value.name == "SHORT"
        first_entry_value = entry_targets_value[0]
        
        if should_check:
            if not isSHORT:
                stat = await findLongOrderStat(stop_loss=stopLoss_value,
                                entry_price=entry_targets_value,
                                symbol=symbol_value,
                                take_profit=take_profit_targets_value,
                                start_timestamp= int(post.date.timestamp() * 1000))
            elif isSHORT:
                stat = await findShortOrderStat(stop_loss=stopLoss_value,
                                entry_price=entry_targets_value,
                                symbol=symbol_value,
                                take_profit=take_profit_targets_value,
                                start_timestamp= int(post.date.timestamp() * 1000))
        else:
            stat = {"tps": [], "entry_reached": [], "stop_loss_reached": None}
       
        print(stat)
        stat['name'] = f'{symbol_value.name}, {position_value.name}'
        allPredicts.append(stat)
        print_colored("********************************************************************************************************************","#0f0")
        tps_length = len(stat['tps'])
        profit_value = 0

        if stat['stop_loss_reached']:
            if tps_length > 0:
                status_value = await sync_to_async(PostStatus.objects.get)(name="FAILED WITH PROFIT", type=tps_length)
            else:
                status_value = await sync_to_async(PostStatus.objects.get)(name="FAILED")
            profit_value = round(((float(stopLoss_value)/float(first_entry_value))-1)*100*float(leverage_value) * (-1 if isSHORT else 1), 5)
        else:
            if tps_length == len(take_profit_targets_value):
                status_value = await sync_to_async(PostStatus.objects.get)(name="FULLTARGET")
                profit_value = round(abs(((take_profit_targets_value[tps_length-1]/first_entry_value)-1)*100*leverage_value), 5)

            elif tps_length > 0:
                status_value = await sync_to_async(PostStatus.objects.get)(name="SUCCESS", type=tps_length)
                profit_value = round(abs(((take_profit_targets_value[tps_length-1]/first_entry_value)-1)*100*leverage_value), 5)
            
        

        # set predict object to DB
        PredictData = {
            "post": post,
            "date": post.date,
            "symbol": symbol_value,
            "position": position_value,
            "market": market_value,
            "leverage": leverage_value,
            "stopLoss": stopLoss_value,
            "margin_mode": marginMode_value,
            "profit": profit_value,
            "status": status_value,  # PENDING = 1
            "order_id": None,
        }
        newPredict = Predict(**PredictData)

        # set entry value objects to DB
        if entry_targets_value:
            entriesLen = len(stat['entry_reached'])
            for i, value in enumerate(entry_targets_value):
                isActive = i < entriesLen 
                date = None
                period = None
                if isActive:
                    time = int(stat['entry_reached'][i]['time'])
                    date = datetime.fromtimestamp(time/ 1000)
                    period = subtractTime(time, int(post.date.timestamp() * 1000))
                entryData = EntryTarget(
                    **{
                        "post": post,
                        "index": i,
                        "value": value,
                        "active": isActive,
                        "period": period,
                        "date": date,
                    }
                )
                await sync_to_async(entryData.save)()
        
        # # set tp value objects to DB
        if take_profit_targets_value:
            tpLen = len(stat['tps'])

            for i, value in enumerate(take_profit_targets_value):
                isActive = i < tpLen 
                date = None
                period = None
                if isActive:
                    time = int(stat['tps'][i]['time'])
                    date = datetime.fromtimestamp(time/ 1000)
                    period = subtractTime(time, int(post.date.timestamp() * 1000))

                takeProfitData = TakeProfitTarget(
                    **{
                        "post": post,
                        "index": i,
                        "value": value,
                        "active": isActive,
                        "period": period,
                        "profit": round(abs(((value/first_entry_value)-1)*100*leverage_value), 5),
                        "date": date,
                    }
                )
                await sync_to_async(takeProfitData.save)()


        await sync_to_async(newPredict.save)()
        return newPredict
    except:
        error_msg.append(model_to_dict(post))
        
        return False
     

## Find message type and save to DB

In [51]:
async def extract_data_from_message(message):
    if isinstance(message, Message):
        is_predict_msg = isPredictMsg(message.message)
        channel = await sync_to_async(Channel.objects.get)(
            channel_id=message.peer_id.channel_id
        )
        PostData = {
            "channel": channel if channel else None,
            "date": message.date,
            "message_id": message.id,
            "message": message.message,
            "reply_to_msg_id": message.reply_to.reply_to_msg_id
            if message.reply_to
            else None,
            "edit_date": message.edit_date,
            "is_predict_msg": is_predict_msg,
        }
        post = Post(**PostData)

        await sync_to_async(post.save)()
        # predict msg
        if is_predict_msg:
            await predictParts(message.message, post)
        return PostData
    else:
        return None



## Get channel history

In [52]:
client = await TelegramClient(username, api_id, api_hash).start()
peer_channel =  PeerChannel(int(config["CHANNEL_ALI_BEY"]))
feyzian_channel = await client.get_entity(peer_channel)

offset_id = 0
limit = 100
all_messages = []
total_messages = 0
total_count_limit = 0
# اول خرداد = 2024/5/21
# end_date = datetime(2024, 5, 21, tzinfo=timezone.utc)
# end_date = datetime(2024, 7, 1, tzinfo=timezone.utc)
# start_date = datetime(2024, 5, 29, tzinfo=timezone.utc)
# اول تیر = 2024/6/21
# start_date = datetime(2024, 6, 21, tzinfo=timezone.utc)
end_date = datetime(2024, 7, 15, tzinfo=timezone.utc)

shouldStop = False
while not shouldStop:
    print("Current Offset ID is:", offset_id, "; Total Messages:", total_messages)
    history = await  client(
    GetHistoryRequest(
            peer=feyzian_channel,
            offset_id=offset_id,
            offset_date=None,
            # offset_date=start_date,
            add_offset=0,
            limit=limit,
            max_id=0,
            min_id=0,
            hash=0,
        )
    )

    if not history.messages:
        break

    messages = history.messages
    for message in tqdm(messages):
        message_date = message.date.replace(tzinfo=timezone.utc)
        # if start_date < message_date:
        #     continue

        if message_date < end_date:
            shouldStop = True
            break
        all_messages.append(message)

    offset_id = messages[len(messages) - 1].id
    total_messages = len(all_messages)
    if total_count_limit != 0 and total_messages >= total_count_limit:
        break

await client.disconnect()

for msg in reversed(all_messages):
    await extract_data_from_message(msg)



# print(error_msg)
convertToJsonFile(error_msg, "error_msg", ".") 
# convertToJsonFile(allPredicts, "allPredicts", ".") 

Current Offset ID is: 0 ; Total Messages: 0


 51%|█████     | 51/100 [00:00<?, ?it/s]


BTC-USDT SHORT
66250 [63000.0, 65300.0] BTC-USDT [62400.0, 61900.0, 61000.0, 60000.0, 59100.0, 58000.0, 56000.0, 54000.0] 1721026500000


{'tps': [{'open': '62443.9', 'close': '62383.6', 'high': '62443.9', 'low': '62356.5', 'volume': '64.20', 'time': 1721042280000}], 'entry_reached': [{'open': '62950.0', 'close': '63064.5', 'high': '63082.6', 'low': '62949.2', 'volume': '129.44', 'time': 1721029380000}, {'open': '64952.3', 'close': '65212.3', 'high': '65326.4', 'low': '64952.2', 'volume': '150.19', 'time': 1721172540000}], 'stop_loss_reached': {'open': '66082.4', 'close': '66330.9', 'high': '66344.7', 'low': '66048.4', 'volume': '838.27', 'time': 1721404920000}}


GRT-USDT SHORT
0.2165 [0.2057, 0.21] GRT-USDT [0.203, 0.202, 0.206, 0.2, 0.195, 0.19] 1721040494000


...

{'tps': [{'open': '0.015598', 'close': '0.015506', 'high': '0.015603', 'low': '0.015486', 'volume': '17517807.00', 'time': 1721585220000}], 'entry_reached': [{'open': '0.016291', 'close': '0.016278', 'high': '0.016315', 'low': '0.016278', 'volume': '1524708.00', 'time': 1721414100000}], 'stop_loss_reached': None}


BTC-USDT SHORT
67210 [65040.0] BTC-USDT [63900.0, 63000.0, 62200.0, 61400.0, 60000.0, 59100.0, 58000.0, 56000.0] 1721414730000


{'tps': [], 'entry_reached': [{'open': '66541.8', 'close': '66594.3', 'high': '66630.8', 'low': '66531.4', 'volume': '29.12', 'time': 1721414760000}], 'stop_loss_reached': {'open': '67167.4', 'close': '67234.9', 'high': '67245.9', 'low': '67119.3', 'volume': '109.78', 'time': 1721417520000}}


BNB-USDT SHORT
664 [595.0, 645.0] BNB-USDT [570.0, 558.0, 540.0, 524.0, 510.0, 495.0, 478.0, 460.0] 1721555114000


{'tps': [], 'entry_reached': [{'open': '594.85', 'close': '595.17', 'high': '595.21', 'low': '594.83', 'volume': '34.41', 'time': 1721571720000}], 'stop_loss_reached': None}
